### Applying any transformation

In this notebook, we will apply some custom transformations on a mesh. By custom filter I mean any numerical manipulation you may want to apply to a 3D object.

Take-home messages :
- You can modify the position of the points of a mesh by considering it as a numpy array
- You can also modify the topological structure if a mesh : pyVista and vedo have different ways of representing the faces of a mesh
- You can modify as you want the pixels/voxels value for an image data
- ...


References
- pyVista user guide data model : https://docs.pyvista.org/version/stable/user-guide/data_model.html
- vedo doc on different geometric objects https://vedo.embl.es/docs/vedo/mesh.html https://vedo.embl.es/docs/vedo/picture.html https://vedo.embl.es/docs/vedo/volume.html

In [1]:
import pyvista
import numpy as np

mesh = pyvista.read("data/amygdala1.vtk").decimate(0.98)  # Load mesh and decimate it

translation = np.array([2, 3, 7])  # Define a translation vector
translated_mesh = mesh.copy()  # Copy the mesh
translated_mesh.points += translation  # Translate the mesh by modifying its points


mesh_with_holes = mesh.copy()  # Copy the mesh
# How triangles are encoded in pyVista ?
# There are stored in a list of integer organized as follow:
# [n0, p0_0, p0_1, ..., p0_n, n1, p1_0, p1_1, ..., p1_n, ...]
# where n0 is the number of points in face 0, and pX_Y is the Y’th point in face X.
# this notation allows to store faces with different number of points.
# We can check that a mesh is only composed of triangles by calling mesh.is_all_triangles()
#
# https://docs.pyvista.org/version/stable/api/core/_autosummary/pyvista.PolyData.faces.html

mesh_with_holes.faces = mesh.faces[
    4:-4
]  # Remove the first and last triangles by removing the first and last 4 integers

# Plot the results
p = pyvista.Plotter(shape=(1, 2))
p.subplot(0, 0)
p.add_mesh(mesh, color="red", opacity=0.6, show_edges=False)
p.add_mesh(translated_mesh, opacity=0.6, show_edges=True)
p.subplot(0, 1)
p.add_mesh(mesh_with_holes, show_edges=True)
p.show()

Widget(value="<iframe src='http://localhost:42537/index.html?ui=P_0x7ff9fc3106d0_0&reconnect=auto' style='widt…

In [2]:
import vedo

vedo.settings.default_backend = "vtk"
import numpy as np

mesh = vedo.load("data/amygdala1.vtk").decimate(0.02)

translation = np.array([2, 3, 7])
translated_mesh = mesh.clone()

points = translated_mesh.points()
translated_mesh.points(points + translation)
# .points() can be used to get or set the points of a mesh


triangles = mesh.faces()
# In vedo the faces are stored as a list of list of integer
# [[p0_0, p0_1, ..., p0_n], [p1_0, p1_1, ..., p1_n], ...]
# where pX_Y is the Y’th point in face X.
# For a triangle mesh, faces is a list of list of 3 integers

reduced_triangles = triangles[
    1:-1
]  # Remove the first and last triangles by removing the first and last elements of triangles
# mesh_with_holes.faces(triangles) <- doesn't work
# .faces() cannot be used to set the faces of a mesh, we must recreate the mesh with the new faces

mesh_with_holes = vedo.Mesh(
    [points + translation, reduced_triangles]
)  # Create a new mesh with the new points and faces


# Plot the results
plotter = vedo.Plotter(shape=(1, 2), axes=0, sharecam=False, interactive=True)
plotter.add(mesh.c("red").alpha(0.6), at=0)
plotter.add(translated_mesh.linewidth(1).alpha(0.6), at=0)
plotter.add(mesh_with_holes.linewidth(1), at=1)
plotter.show()

In [5]:
image = pyvista.read("data/Tux.png")  # Load the image
image

UniformGrid (0x7ffa2d9e51c0)
  N Cells:      82632
  N Points:     83210
  X Bounds:     0.000e+00, 2.640e+02
  Y Bounds:     0.000e+00, 3.130e+02
  Z Bounds:     0.000e+00, 0.000e+00
  Dimensions:   265, 314, 1
  Spacing:      1.000e+00, 1.000e+00, 1.000e+00
  N Arrays:     1

In [3]:
image = pyvista.read("data/Tux.png")  # Load the image

# Get the array names
# An image is a UniformGrid with a single array
# The array contains the color of each pixel
print("Arrays names are : {}".format(image.array_names))
image.get_array("PNGImage").shape

new_image = image.copy()  # Copy the image

new_image.get_array("PNGImage")[:, 0:3] = (
    255 - image.get_array("PNGImage")[:, 0:3]
)  # Invert the colors
# The last dimension of the array is the transparency

# Plot the results
p = pyvista.Plotter(shape=(1, 2))
p.subplot(0, 0)
p.add_mesh(image, rgb=True)
p.subplot(0, 1)
p.add_mesh(new_image, rgb=True)
p.show()

Arrays names are : ['PNGImage']


Widget(value="<iframe src='http://localhost:42537/index.html?ui=P_0x7ffa5c5025e0_1&reconnect=auto' style='widt…